## Eliza implementation

This notebook is taken from: https://github.com/itu-qsp/eliza

## 1. Loading and running Eliza

We adapted the Eliza code in the *eliza.py* file from the original Github so that the Speaker information and turn identifier are saved in a JSON structure for each turn. We wil use the stored conversation to create a Pandas frame and try to assign emotions to each utterance.

In [1]:
import eliza as el
import pandas as pd

In [9]:
el.talk_to_me()

My name is Eliza. What is your name?


>  Piek


Hello Piek. How are you feeling today?


>  Stop


We can now inspect the stored conversation and load the JSON in a Pandas dataframe.

In [5]:
print(el.conversation)

[{'utterance': 'Hello Piek. How are you feeling today?', 'speaker': 'Eliza', 'turn_id': 1}, {'utterance': 'I am sad', 'speaker': 'Piek', 'turn_id': 1}, {'utterance': 'How do you feel about being sad?', 'speaker': 'Eliza', 'turn_id': 1}, {'utterance': 'Bad', 'speaker': 'Piek', 'turn_id': 1}, {'utterance': 'How do you feel when you say that?', 'speaker': 'Eliza', 'turn_id': 1}, {'utterance': 'I feel better when I talk about it', 'speaker': 'Piek', 'turn_id': 1}, {'utterance': 'Do you often feel better when you talk about it?', 'speaker': 'Eliza', 'turn_id': 1}, {'utterance': 'yes', 'speaker': 'Piek', 'turn_id': 1}, {'utterance': 'I see.', 'speaker': 'Eliza', 'turn_id': 1}, {'utterance': 'My cat died yesterday', 'speaker': 'Piek', 'turn_id': 1}, {'utterance': 'Why do you say that your cat died yesterday?', 'speaker': 'Eliza', 'turn_id': 1}, {'utterance': 'It made me sad', 'speaker': 'Piek', 'turn_id': 1}, {'utterance': 'Why do you say that it made you sad?', 'speaker': 'Eliza', 'turn_id':

In [6]:
df = pd.DataFrame.from_dict(el.conversation)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   utterance  21 non-null     object
 1   speaker    21 non-null     object
 2   turn_id    21 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 632.0+ bytes


In [7]:
df.head()

,utterance,speaker,turn_id
0,Hello Piek. How are you feeling today?,Eliza,1
1,I am sad,Piek,1
2,How do you feel about being sad?,Eliza,1
3,Bad,Piek,1
4,How do you feel when you say that?,Eliza,1


# 2. Annotate the conversation with emotion labels

We define a simple loop over the turns in the conversation to add an Ekman emotion label:

In [5]:
file = 'conversation.csv'
df = pd.read_csv(file)

In [ ]:
ekman_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
gold_labels = []
gold = ""
for index, utterance in enumerate(df['utterance']):
    print(utterance)
    while not gold in ekman_labels:
        gold = input("label> ")
    gold_labels.append(gold)
    gold=""

print(gold_labels)
df['Gold']=gold_labels

Hello Piek. How are you feeling today?


label>  neutral


I am sad


label>  sadness


How do you feel about being sad?


...

I feel better when I talk about it


label>  joy


Do you often feel better when you talk about it?


label>  neutral


yes


# 3. Save the conversation to disk

You need to annotated conversation for the final assignment. We therefore save it to disk so that we can load it later.

In [8]:
file = "my_emotional_conversation.csv"
df.to_csv(file)

## End of Notebook